In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

model_path = "/content/drive/MyDrive/distilbert-review-classifier/distilbert-review-classifier"

model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)

In [ ]:
df_new = pd.read_csv("gold_labeled_subset.csv")   # your clean gold set or combined set
dataset = Dataset.from_pandas(df_new)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=16,
    num_train_epochs=3,
    learning_rate=3e-5,
    weight_decay=0.01,
    save_total_limit=1,
    logging_strategy="epoch",   # log once per epoch
    eval_strategy="no", # optional, evaluate per epoch
    report_to="none",     # <--- disables wandb
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

# 5. Continue training
trainer.train()

In [ ]:
save_path = "/content/drive/MyDrive/distilbert-review-classifier-v2"

trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)